In [ ]:
%matplotlib notebook
from collections import Counter
from pprint import pprint
import importlib
import os
import sys

from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from mrcnn.model import MaskRCNN
from skimage.color import label2rgb
from skimage.io import imread, imsave
from skimage.transform import resize
import imgaug.augmenters as iaa
import keras.backend as K
import matplotlib.pyplot as plt
import numpy as np

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Setup Data

In [ ]:
def setup_args():
    from keras.applications.resnet50 import preprocess_input
    from bidict import bidict
    from imgaug import augmenters as iaa
    from imgaug.parameters import Normal, Discretize
    from skimage.transform import resize
    
    def load_config(path):
        spec = importlib.util.spec_from_file_location(
            "maskrcnn_config", path)
        config_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(config_module)
        return config_module.Config()

#     def preprocess_data(image):
#         '''Transform the image before (possibly caching) and input to the network.'''
#        # This is done automatically by MRCNN

    def postprocess_data(image):
        '''Inverse transform of preprocess_data, used when trying to visualize images out of the dataset.'''
        return (image).astype(np.uint8)

    def pipeline(gen, aug_config=None):
        '''The pipeline to run the dataset generator through.'''
        from abyss_deep_learning.keras.classification import onehot_gen, augmentation_gen

        return gen 
#                 (
#             augmentation_gen(
#                 onehot_gen(gen, num_classes=args['num_classes'])
#             , aug_config, enable=(aug_config is not None))
#         )

    augmentation_config = iaa.Sequential([ 
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.Affine(
            scale=(0.8, 1.2),
            translate_percent=(-0.2, 0.2), 
            rotate=(-22.5, 22.5),
            mode='constant', cval=0, order=0
        ),
        
#         iaa.Sequential([ # Colour aug
#             iaa.ChangeColorspace(from_colorspace="RGB", to_colorspace="HSV"),
#             iaa.WithChannels(0, iaa.Add(Discretize(Normal(0, 256 / 10)))),
#             iaa.WithChannels(1, iaa.Add(Discretize(Normal(0, 256 / 5)))),
#             iaa.WithChannels(2, iaa.Add(Discretize(Normal(0, 256 / 5)))),
#             iaa.ChangeColorspace(from_colorspace="HSV", to_colorspace="RGB")
#         ])
    ])

    args = {
        'augmentation': augmentation_config,    # Training augmentation
#         'caption_map': caption_map,             # Captio
        'data': {
            'base_dir': "/data/abyss/oceaneering/annotations",
            'name': "separation",
            'sets': ('train', 'val')
        },
        'config': load_config('/home/docker/src/abyss/deep-learning/configs/oceaneering2.py'),
        'image_dims': (512, 512, 3),    # What to resize images to before CNN
        'nn_dtype': np.float32,         # Pretrained networks are in float32
        'num_classes': None,            # Calculate later
#         'use_balanced_set': False,      # Force the use of the largest class-balanced dataset
#         'use_cached': False,            # Cache the dataset in memory
#         'use_class_weights': True,      # Use class population to weight in the training loss
#         'use_parallel': False,          # Use multiple GPUs
#         'preprocess_data': preprocess_data,
        'postprocess_data': postprocess_data,
        'pipeline': pipeline
    }
    args['num_classes'] = args['config'].NUM_CLASSES
    
    return args
ARGS = setup_args()

# Setup Datasets

In [ ]:
def setup_datasets(args):
    from abyss_deep_learning.datasets.coco import MaskRcnnInstSegDataset
    
    dataset = dict()
    for set_name in args['data']['sets']:
        path = os.path.join(args['data']['base_dir'], "{:s}/{:s}.json".format(args['data']['name'], set_name))
        dataset[set_name] = MaskRcnnInstSegDataset(
            path, ARGS['config'])
        print("\n", set_name)
#         dataset[set_name].print_class_stats()

    print("\nNumber of classes:", args['num_classes'])
    cats = dataset['train'].coco.loadCats(dataset['train'].coco.getCatIds())
    class_names = ["BG"] + [
        cat['name'] for cat in sorted(cats, key=lambda x: x['id'])]
    print(class_names)
    return dataset, class_names

DATASET, ARGS['class_names'] = setup_datasets(ARGS)

In [ ]:
def display_from_inputs(inputs, **kwargs):
    from mrcnn.visualize import display_instances
    from mrcnn.utils import unmold_mask
    print(inputs[4].shape)
    N = np.argwhere(inputs[4][0] == 0)[0][0]
    image, image_meta = inputs[0][0], inputs[1][0]
    rpn_match, rpn_bbox = inputs[2][0], inputs[3][0]
    gt_class_ids, gt_boxes, gt_masks = inputs[4][0, :N], inputs[5][0, :N], inputs[6][0, ..., :N]

    masks = np.array([
        unmold_mask(gt_masks[..., idx], gt_boxes[idx], image.shape)
        for idx in range(N)]).transpose([1, 2, 0])

    display_instances(
        ARGS['postprocess_data'](image), gt_boxes, masks, gt_class_ids, ARGS['class_names'], **kwargs)
        
def view_dataset_samples(num_rows=2):
    plt.figure()
    print("Column-wise left to right, bottom row:")
    for i, (name, ds) in enumerate(DATASET.items()):
        print(name, end=' ')
        for j, (inputs, targets) in enumerate(ARGS['pipeline'](
            ds.mrcnn_generator(shuffle=True, augmentation=ARGS['augmentation']))):
            ax = plt.subplot(num_rows, 3, 3 * j + i + 1)
            print({
                k: (kk.dtype, kk.shape)
                for k, kk in enumerate(inputs)
            })
            display_from_inputs(inputs, ax=ax)
#             plt.title(', '.join([ARGS['caption_map'].inv[int(cap_id)] for cap_id in np.argwhere(label)]))
            plt.axis('off')
            if j + 1 == num_rows:
                break

view_dataset_samples(num_rows=2)

# Setup Model

In [ ]:
class Experiment(object):
    def __init__(self, config, model_dir):
        self.epoch = 0
        self.model = None
        self.config = config
        self.model_dir = model_dir
        self.compiled = False
    
    def create(self, model_path=None, train=False, fresh_heads=False, gpu_count=1):
        if model_path is None:
            model_path = '/data/models/mask_rcnn_coco.h5'
        elif model_path is False:
            model_path = None
#         if not train:
        self.config.IMAGES_PER_GPU = 1
        self.config.BATCH_SIZE = 1
        self.model = None
        K.clear_session()
        self.config.GPU_COUNT = gpu_count
        self.model = MaskRCNN(
            mode=("training" if train else "inference"),
            config=self.config, model_dir=self.model_dir)
        if model_path: 
            exclude = [
                "mrcnn_class_logits", "mrcnn_bbox_fc",
                "mrcnn_bbox", "mrcnn_mask"] if fresh_heads else []
            self.model.load_weights(model_path, by_name=True, exclude=exclude)
    
    def train(self, learning_rate, epochs, layers, **kwargs):
        return self.model.train(
            DATASET['train'], DATASET['val'], 
            learning_rate, epochs, layers,
            **kwargs
        )


# Train MRCNN heads

In [ ]:
model_path = False # None for COCO pretrained weights, False for empty
logdir = os.path.join("/data/log/maskrcnn/broccoli-allages/{:s}".format(ARGS['data']['name']))
!mkdir -p "$logdir"
best_path = os.path.join(logdir, "models/best.{epoch:03d}-{val_loss:.4f}.h5")

In [ ]:
config = ARGS['config']
# config.USE_MINI_MASK = True
# config.BATCH_SIZE = 1
# config.IMAGES_PER_GPU = 1
# config.WEIGHT_DECAY = 1e-4
# config.VALIDATION_STEPS = len(DATASET['val'].data_ids) // config.BATCH_SIZE
config.display()

In [ ]:
exp = None
exp = Experiment(ARGS['config'], logdir)
model = exp.create(model_path=model_path, train=True, fresh_heads=True, gpu_count=1)

In [ ]:
train_layers = 'heads'
exp.train(
    1e-6, 50, train_layers,
    custom_callbacks=[EarlyStopping(patience=2, min_delta=0.05, verbose=1)],
    augmentation=None,#ARGS['augmentation'],
    no_augmentation_sources=None)
exp.model.keras_model.save_weights(os.path.join(logdir, 'heads.h5'))

In [ ]:
saved_model_path = os.path.join(logdir, 'heads.h5')

exp = None
exp = Experiment(ARGS['config'], logdir)
model = exp.create(model_path=saved_model_path, train=True, fresh_heads=False)

if exp.model.epoch == 0:
    exp.model.epoch = 12

callbacks = [
    ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=3, cooldown=10, verbose=1),
    EarlyStopping(
        monitor='val_loss', min_delta=0.0, patience=20, verbose=1, mode='auto'),
    ModelCheckpoint(
        os.path.join(logdir, "weights.{epoch:02d}-{val_loss:.2f}.hdf5"),
        verbose=1, save_best_only=True, save_weights_only=True)
]
# try:
#     lr = K.get_value(exp.model.keras_model.optimizer.lr)
# except AttributeError:
lr = 1e-3


exp.train(
    lr, 200, 'all',
    augmentation=ARGS['augmentation'],
    custom_callbacks=callbacks,
    no_augmentation_sources=None)
exp.model.keras_model.save_weights(os.path.join(logdir, 'final2.h5'))

In [ ]:
print(1)

In [ ]:
exp = None
config.USE_MINI_MASK = False
config.IMAGES_PER_GPU = 1
exp = Experiment(ARGS['config'], logdir)
model = exp.create(model_path=os.path.join(logdir, 'final2.h5'), train=False, fresh_heads=False)

# Visualisation

In [ ]:
from mrcnn.utils import expand_mask
from mrcnn.visualize import display_images, display_instances
# from abyss_deep_learning.keras.segmentation import jaccard_index

def plot_test(gen, model, num_images=1, show=False):
    from scipy.optimize import linear_sum_assignment
    ious_list = []
    i = 0
    for ((images, image_meta, rpn_match, rpn_bbox, gt_class_ids, gt_boxes, gt_masks), targets) in gen:
        image = images[0]
        valid = np.all(gt_boxes[0], axis=1)
        class_ids = gt_class_ids[0, valid]
        masks = gt_masks[0, ..., valid].transpose((1, 2, 0))
        boxes = gt_boxes[0, valid, ...]
        
        labels = expand_mask(boxes, masks, image.shape).astype(np.uint8)
        r = model.detect([image], verbose=True)[0]
        num_pred = len(r['class_ids'])
        
        num_gt = len(class_ids)
        print("GTs = {:d}, Pred = {:d}".format(num_gt, num_pred))
        ious = np.array([[
            1 #jaccard_index(r['masks'][..., i] , labels[..., j]) 
                for j in range(labels.shape[-1])] 
                for i in range(r['masks'].shape[-1])])
        pred_idx, gt_idx = linear_sum_assignment(1-ious)
        r['ious'] = np.array([ious[pred_idx[i], gt_idx[i]] 
                              if (i in pred_idx and i in gt_idx) else 0.0 for i in range(num_pred)])
        print("IoUs", r['ious'])
        print("Scores", r['scores'])
        ious_list.append(ious)
        class_names = ['BG'] + [cat['name'] for cat in DATASET['train'].coco.cats.values()]
        if show:
            plt.figure()
            ax = plt.subplot(1, 2, 1)
            display_instances(
                image + ARGS['config'].MEAN_PIXEL,
                boxes,
                masks,
                class_ids,
                class_names, ax=ax)
            ax = plt.subplot(1, 2, 2, sharex=ax, sharey=ax)
            display_instances(
                image + ARGS['config'].MEAN_PIXEL,
                r['rois'],
                r['masks'],
                r['class_ids'],
                class_names, ax=ax)
            
#         imsave("/tmp/maskrcnn/image.png", (image + config.MEAN_PIXEL).astype(np.uint8))
        i += 1    
        if i >= num_images:
                break
    return ious_list

ious = plot_test(
    DATASET['val'].mrcnn_generator(shuffle=True),
    exp.model, num_images=1, show=True)

In [ ]:
import mrcnn.visualize as viz
# evaluate_coco(model, dataset_val, coco_val, eval_type="segm", limit=0, image_ids=None)
viz.display_weight_stats(exp.model)

In [ ]:
coco = DATASET['val']
image = coco.load_image(1)
exp
# Get activations of a few sample layers
activations = exp.model.run_graph([image], [
#     ("input_image",        exp.model.keras_model.get_layer("input_image").output),
    ("res2c_out",          exp.model.keras_model.get_layer("res2c_out").output),
    ("res3c_out",          exp.model.keras_model.get_layer("res3c_out").output),
    ("res4c_out",          exp.model.keras_model.get_layer("res4c_out").output),
    ("res5c_out",          exp.model.keras_model.get_layer("res5c_out").output),
    ("rpn_bbox",           exp.model.keras_model.get_layer("rpn_bbox").output),
    ("roi",                exp.model.keras_model.get_layer("ROI").output),
])

plt.figure()
layer_names = ["res2c_out", "res3c_out", "res4c_out", "res5c_out"]
ax = None
for i, layer in enumerate(layer_names):
    ax = plt.subplot(len(layer_names) // 2, 2, i + 1)
    plt.imshow(activations[layer].sum(axis=3)[0])
    plt.title(layer)
plt.tight_layout()

In [ ]:
# # Backbone feature map
# display_images(np.transpose(activations["res2c_out"][0,:,:,:4], [2, 0, 1]), cols=4)
# display_images(np.transpose(activations["res3c_out"][0,:,:,:4], [2, 0, 1]), cols=4)
# display_images(np.transpose(activations["res4c_out"][0,:,:,:4], [2, 0, 1]), cols=4)
# display_images(np.transpose(activations["res5c_out"][0,:,:,:4], [2, 0, 1]), cols=4)
